In [7]:
from __future__ import division
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from sklearn import preprocessing

In [8]:
#data = pd.read_csv('ausgrid/customer_1.csv', index_col=0, sep=";")
data = pd.read_csv('../data/load/dataid171.csv', index_col=0, sep=";")
data.loc[data['loadProd'] < 0, 'loadProd'] = 0
data.head()

,date,time,dateTime,dayOfWeek,holiday,dayId,loadCons,loadProd,loadGrid
id,,,,,,,,,
1,2014-01-01,00:00:00,2014-01-01 00:00:00,3,1,1,0.885267,0.0,0.885267
2,2014-01-01,01:00:00,2014-01-01 01:00:00,3,1,1,0.512767,0.0,0.512767
3,2014-01-01,02:00:00,2014-01-01 02:00:00,3,1,1,0.448867,0.0,0.448867
4,2014-01-01,03:00:00,2014-01-01 03:00:00,3,1,1,0.473483,0.0,0.473483
5,2014-01-01,04:00:00,2014-01-01 04:00:00,3,1,1,0.440150,0.0,0.440150


In [9]:
gen = pd.DataFrame(data[['loadProd']])
con = pd.DataFrame(data[['loadCons']])
con = con['loadCons']
gen = gen['loadProd']

In [10]:
def Fun(D, sol):
    val = 0.0
    for i in range(D):
        val = val + sol[i] * sol[i]
    return val

# function we are attempting to optimize (minimize)
def funcG(xg):
    dim = len(x)
    bat_actions = []
    grid_actions = []
    bat_actions.append(x[0]-bat_t0)
    grid_actions.append(demand[0]-pv_gen[0]+bat_actions[0])
    bat_action = x[0]-bat_t0
    grid_sum=demand[0]-pv_gen[0]+bat_action
    for i in range(1,dim):
        bat_actions.append(x[i]-x[i-1]) 
        grid_actions.append(demand[i]-pv_gen[i]+bat_actions[i])
        bat_action = x[i]-x[i-1]
        grid_sum+=demand[i]-pv_gen[i]+bat_action
    print("pos ",x)
    print("bat_actions ",bat_actions)
    print("demand ",demand)
    print("pv_gen ",pv_gen)
    print("grid_actions ",grid_actions)
    print("grid_sum ",grid_sum)
    print("bat_t0 ",bat_t0)
    print()
    return xg[0]
        
def cost_function(x,demand,pv_gen,bat_t0,elprices):
    dim = len(x)
    bat_action = x[0]-bat_t0
    grid_sum=demand[0]-pv_gen[0]+bat_action
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        grid_sum+=demand[i]-pv_gen[i]+bat_action
    return grid_sum

def cost_function2(x,demand,pv_gen,bat_t0,elprices):
    dim = len(x)
    bat_action = x[0]-bat_t0
    grid_sum=abs(demand[0]-pv_gen[0]+bat_action)
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        grid_sum+=abs(demand[i]-pv_gen[i]+bat_action)
    return grid_sum

def cost_function22(x,demand,pv_gen,bat_t0,elprices):
    dim = len(x)
    bat_action = x[0]-bat_t0
    if bat_action>5:
        bat_action=5
        x[0]=bat_t0+5
    if bat_action<-5:
        bat_action=-5
        x[0]=bat_t0-5
    grid_sum=abs(demand[0]-pv_gen[0]+bat_action)
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        if bat_action>5:
            bat_action=5
            x[i]=x[i-1]+5
        if bat_action<-5:
            bat_action=-5
            x[i]=x[i-1]-5
        grid_sum+=abs(demand[i]-pv_gen[i]+bat_action)
    return grid_sum

def cost_function3(x,demand,pv_gen,bat_t0,elprices):
    dim = len(x)
    bat_action = x[0]-bat_t0
    grid_sum=demand[0]-pv_gen[0]+bat_action
    grid_sum=grid_sum*elprices[0]
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        grid_sum+=((demand[i]-pv_gen[i]+bat_action)*elprices[i])
    return grid_sum

def cost_function4(x,demand,pv_gen,bat_t0,bat_act0):
    batActions = []
    score = 0
    dim = len(x)
    bat_action = x[0]-bat_t0
    batActions.append(bat_action)
    grid_sum=abs(demand[0]-pv_gen[0]+bat_action)
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        batActions.append(bat_action)
        grid_sum+=abs(demand[i]-pv_gen[i]+bat_action)
        
    if bat_act0==1:
        if batActions[0]>0:
            score+=batActions[0]
        elif batActions[0]<0:
            if bat_t0<13:
                score-=1
            score+=abs(batActions[0])
            bat_act0=-1
    elif bat_act0==-1:
        if batActions[0]<0:
            score+=abs(batActions[0])
        elif batActions[0]>0:
            if bat_t0>0:
                score-=1
            score+=batActions[0]
            bat_act0=1
            
    for i in range(1,dim):
        if bat_act0==1:
            if batActions[i]>0:
                score+=batActions[i]
            elif batActions[i]<0:
                if x[i-1]<13:
                    score-=1
                score+=abs(batActions[i])
                bat_act0=-1
        elif bat_act0==-1:
            if batActions[i]<0:
                score+=abs(batActions[i])
            elif batActions[i]>0:
                if x[i-1]>0:
                    score-=1
                score+=batActions[i]
                bat_act0=1
    grid_sum-=(score*0.8)
    return (grid_sum),bat_act0

In [11]:
class Battery:
    def __init__(self,cap,soc,act):
        self.capacity=cap
        self.SOC=soc
        self.action=act

In [12]:
class BatAlgorithm2():
    def __init__(self, D, NP, N_Gen, A, r, Qmin, Qmax, Lower, Upper, function,demand,pv_gen,bat_t0,elprices):
        self.D = D  #dimension
        self.NP = NP  #population size 
        self.N_Gen = N_Gen  #generations
        self.A = A  #loudness
        self.r = r  #pulse rate
        self.Qmin = Qmin  #frequency min
        self.Qmax = Qmax  #frequency max
        self.Lower = Lower  #lower bound
        self.Upper = Upper  #upper bound

        self.f_min = 0.0  #minimum fitness
        
        self.Lb = [0] * self.D  #lower bound
        self.Ub = [0] * self.D  #upper bound
        self.Q = [0] * self.NP  #frequency

        self.v = [[0 for i in range(self.D)] for j in range(self.NP)]  #velocity
        self.Sol = [[0 for i in range(self.D)] for j in range(self.NP)]  #population of solutions
        self.Fitness = [0] * self.NP  #fitness
        self.best = [0] * self.D  #best solution
        self.Fun = function
        
        self.demand = demand  #som pridal
        self.pv_gen = pv_gen  #som pridal
        self.elprices = elprices   #som pridal
        self.bat_t0 = bat_t0  #som pridal
        self.grid_actions = []#som pridal
        self.demand_sum = 0   #som pridal
        self.grid_total = 0   #som pridal
        
        #print(demand)
        #print(pv_gen)
        #print(bat_t0)


    def best_bat(self):
        i = 0
        j = 0
        for i in range(self.NP):
            if self.Fitness[i] < self.Fitness[j]:
                j = i
        for i in range(self.D):
            self.best[i] = self.Sol[j][i]
        self.f_min = self.Fitness[j]

    def init_bat(self):
        for i in range(self.D):
            self.Lb[i] = self.Lower
            self.Ub[i] = self.Upper

        for i in range(self.NP):
            self.Q[i] = 0
            for j in range(self.D):
                rnd = np.random.uniform(0, 1)
                self.v[i][j] = 0.0
                self.Sol[i][j] = self.Lb[j] + (self.Ub[j] - self.Lb[j]) * rnd
            #self.Fitness[i] = self.Fun(self.D, self.Sol[i])                                    #som zmenil
            self.Fitness[i] = self.Fun(self.Sol[i],self.demand,self.pv_gen,self.bat_t0,self.elprices)         #som zmenil
        self.best_bat()

    def simplebounds(self, val, lower, upper):
        if val < lower:
            val = lower
        if val > upper:
            val = upper
        return val

    def move_bat(self):
        S = [[0.0 for i in range(self.D)] for j in range(self.NP)]

        self.init_bat()

        for t in range(self.N_Gen):
            for i in range(self.NP):
                rnd = np.random.uniform(0, 1)
                self.Q[i] = self.Qmin + (self.Qmax - self.Qmin) * rnd
                for j in range(self.D):
                    self.v[i][j] = self.v[i][j] + (self.Sol[i][j] -
                                                   self.best[j]) * self.Q[i]
                    S[i][j] = self.Sol[i][j] + self.v[i][j]

                    S[i][j] = self.simplebounds(S[i][j], self.Lb[j],
                                                self.Ub[j])

                rnd = np.random.random_sample()

                if rnd > self.r:
                    for j in range(self.D):
                        S[i][j] = self.best[j] + 0.001 * random.gauss(0, 1)
                        S[i][j] = self.simplebounds(S[i][j], self.Lb[j],
                                                self.Ub[j])
                        
                #Fnew = self.Fun(self.D, S[i])                                           #som zmenil
                Fnew = self.Fun(S[i],self.demand,self.pv_gen,self.bat_t0,self.elprices)                #som zmenil

                rnd = np.random.random_sample()

                if (Fnew <= self.Fitness[i]) and (rnd < self.A):
                    for j in range(self.D):
                        self.Sol[i][j] = S[i][j]
                    self.Fitness[i] = Fnew

                if Fnew <= self.f_min:
                    for j in range(self.D):
                        self.best[j] = S[i][j]
                    self.f_min = Fnew

        #som pridal
        bat_actions = []                                                #som pridal
        self.grid_actions = []
        bat_actions.append(self.best[0]-self.bat_t0)
        self.grid_actions.append(self.demand[0]-self.pv_gen[0]+bat_actions[0])
        #pv_actions.append()
        self.demand_sum = self.demand[0]
        self.grid_total = self.grid_actions[0]
        for i in range(1,self.D):
            bat_actions.append(self.best[i]-self.best[i-1])
            self.grid_actions.append(self.demand[i]-self.pv_gen[i]+bat_actions[i])
            self.demand_sum+=self.demand[i]
            self.grid_total+=self.grid_actions[i]
        #print(self.f_min)
        #print(self.best)
        bat.SOC=self.best[self.D-1]
        #print()

In [ ]:
# For reproducive results
#random.seed(5)

for i in range(10):
    Algorithm = BatAlgorithm2(10, 40, 1000, 0.5, 0.5, 0.0, 2.0, -10.0, 10.0, Fun)
    Algorithm.move_bat()

In [21]:
windowSize=24
start=1#3626#1
#end=18*7
end = 48#3700#8761
bat=Battery(13, 0, 1)
total_demand = 0
total_grid = 0
import time
zac = int(round(time.time() * 1000))
print(zac)
gridTotalProgress = []
batSOCProgress = []
batSOCProgress.append(0)

for c in range(start,end,windowSize):
    d=c+windowSize
    initial= []
    genWindow = []
    conWindow = []
    BSSBounds = []
    elprices = []
    for i in range(c,d):
        if i<=end:
            hour = i-3626
            hour = hour % 24
            if hour >=0 and hour <= 6:
                elprices.append(0.005)
            if hour >=7 and hour <= 14:
                elprices.append(0.057)
            if hour >=15 and hour <= 20:
                elprices.append(0.09)
            if hour >=21 and hour <= 22:
                elprices.append(0.057)
            if hour >=23 and hour <= 24:
                elprices.append(0.005)
            genWindow.append(gen[i])
            conWindow.append(con[i])
            BSSBounds.append((0,bat.capacity))
            if i % 10 == 0:
                print(i)     
    Algorithm = BatAlgorithm2(len(genWindow), 1000, 2000, 0.5, 0.5, 0.0, 2.0, 0.0, bat.capacity, cost_function22, conWindow, genWindow, bat.SOC, elprices)
    Algorithm.move_bat()
    
    l = len(Algorithm.grid_actions)
    for j in range(0,l):
        gridTotalProgress.append(Algorithm.grid_actions[j])
        batSOCProgress.append(Algorithm.best[j])
    total_demand+=Algorithm.demand_sum
    total_grid+=Algorithm.grid_total
print('FINAL:')
print('Total demand: ',total_demand)
print('Total grid: ',total_grid)
                
    #print(c, genWindow)
    #psoOpt = PSO(cost_function,initial,BSSBounds,num_particles=1000,maxiter=100,demand=conWindow,pv_gen=genWindow,bat_t0=bat.SOC)
    #l = len(psoOpt.grid_actions)
    
kon = int(round(time.time() * 1000))
print(kon)
print(kon-zac)

1586363941583
10
20
30
40
FINAL:
Total demand:  25.657433333333323
Total grid:  3.779428873159964
1586364480850
539267


In [22]:

g=0
gi=0
ge=0

b=0
bc=0
bd=0

for i in(gridTotalProgress):
    if i >= 0:
        gi=gi+i
    if i < 0:
        ge=ge-i
    g=g+i

l = len(batSOCProgress)
print('len ', l)
for i in range(1,l):
    action = batSOCProgress[i]-batSOCProgress[i-1]
    if action > 0:
        bc=bc+action
    if action <= 0:
        bd=bd-action
    b=b+action
        
print(g)
print(gi)
print(ge)

print()

#print(batSOCProgress)
print(b)
print(bc)
print(bd)#111.2 85.49

#np.savetxt("batVymenaGridRokDom171.csv", gridTotalProgress, delimiter=",")
#np.savetxt("batVymenaBatRokDom171.csv", batSOCProgress, delimiter=",")

len  49
3.779428873159966
13.505189731454697
9.725760858294736

4.44991220649329
22.623004304155188
18.173092097661897


In [11]:
g=0
gi=0
ge=0

b=0
bc=0
bd=0

for i in(gridTotalProgress):
    if i >= 0:
        gi=gi+i
    if i < 0:
        ge=ge-i
    g=g+i

l = len(batSOCProgress)
print('len ', l)
for i in range(1,l):
    action = batSOCProgress[i]-batSOCProgress[i-1]
    if action > 0:
        bc=bc+action
    if action <= 0:
        bd=bd-action
    b=b+action
        
print(g)
print(gi)
print(ge)

print()

#print(batSOCProgress)
print(b)
print(bc)
print(bd)#111.2 85.49

np.savetxt("batVymenaGridRokDom93.csv", gridTotalProgress, delimiter=",")
np.savetxt("batVymenaBatRokDom93.csv", batSOCProgress, delimiter=",")

len  76
-4.427732903016555
6.433979398737988
10.861712301754544

4.075550430316763
36.13618962558162
32.06063919526486


In [38]:
np.savetxt("batVymenaGridRokDom93.csv", gridTotalProgress, delimiter=",")
np.savetxt("batVymenaBatRokDom93.csv", batSOCProgress, delimiter=",")

In [91]:
dil = [[3.0 for i in range(10)] for j in range(40)]
dil[33]

[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]

In [25]:
c=[0] * 10
c

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]